In [1]:
# tampering with the kwd file on passaro
# the /experiment folder is on the ssd of passaro (/usr/local/experiment/raw_data)
%matplotlib inline
from __future__ import division
import pdb
import sys
import pandas as pd
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib
import scipy.signal as sg
import math
import scipy as sp
import socket
import os
import wave
import struct
import h5py
from scipy.signal import hilbert
import glob
import errno    
import os
import shutil as sh
import yaml
import logging
#from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

# Check wihic computer to decide where the things are mounted
comp_name=socket.gethostname()
logger.info('Computer: ' + comp_name)
if  comp_name == 'chim':
    #sys.path.append('/Users/zeke/experiment/ephysDataManagement/ephysScripts')
    experiment_folder = os.path.join(os.path.abspath('W:') , "earneodo", "bci_zf")
elif  'lookfar' in comp_name:
    repos_folder = os.path.abspath('/Users/zeke/reposs')
    experiment_folder = os.path.join(os.path.abspath('/Volumes'), 
                                     'gentner', 
                                     'earneodo', 
                                     'bci_zf')

elif 'lintu' in comp_name:
    repos_folder = os.path.abspath('/mnt/cube/earneodo/repos')
    experiment_folder = os.path.join('/mnt/cube/earneodo/bci_zf/')

elif 'passaro' in comp_name:
    repos_folder = os.path.abspath('/mnt/cube/earneodo/repos')
    cube_experiment_folder = os.path.join('/mnt/cube/earneodo/bci_zf/')
    store_experiment_folder = os.path.join('/Data/bci_zf/')
    experiment_folder = os.path.join('/usr/local/experiment')

sys.path.append(os.path.join(repos_folder, 'soundflow', 'sound_tools'))
sys.path.append(os.path.join(repos_folder, 'ephysflow'))
sys.path.append(os.path.join(repos_folder, 'swissknife'))

#import soundtools as st
from scipy.io import wavfile
from swissknife.bci.core import expstruct as et
from swissknife.bci import supersession as ss
from swissknife.bci import stimalign as sa

logger.info('All modules loaded')

#for the kilosort
import subprocess
import paramiko
import select


sys.path.append(os.path.join(repos_folder, 'ephysflow', 'bci_pipeline'))
from swissknife.bci.core.file import h5_functions as h5f
from swissknife.bci.core import kwik_functions as kwkf
from swissknife.bci.core.pipeline.core import kilosort

#logger = logging.getLogger('sort_functions')

def ssh_run(host, cmd):
    #logger.info('Will run ssh command {} on {}'.format(host, cmd))
    ssh = subprocess.Popen(["ssh", "%s" % host, '%s;' % cmd, 'exit;'],
                      shell=False,
                      stdout=subprocess.PIPE,
                      stderr=subprocess.PIPE)

    result = ssh.stdout.readlines()
    if result == []:
        error = ssh.stderr.readlines()
        print >>sys.stderr, "ERROR: %s" % error
        #logger.error('something went wrong')
    else:
        print (result)
        #logger.info('finished')    
    return (result)

def ssh_paramiko(host, cmd):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host)
    print("Connected to %s" % host)
    stdin, stdout, stderr = ssh.exec_command(cmd)
    msg = []
    while not stdout.channel.exit_status_ready():
        # Only print data if there is data to read in the channel
        if stdout.channel.recv_ready():
            rl, wl, xl = select.select([stdout.channel], [], [], 0.0)
            if len(rl) > 0:
                # Print data from stdout
                print (msg.append(stdout.channel.recv(1024)))
    print("finished command, returning")
    ssh.close()
    return msg

# Make one single binary file
def make_binary(bird, sess):
    print('Making binary')
    fn = et.file_names(bird, sess)
    params = et.get_parameters(bird, sess)
    kwd_path = et.file_path(fn, 'ss', 'ss_raw')
    bin_path = et.file_path(fn, 'ss', 'ss_bin')
    chan_list = params['channel_config']['neural']
    h5f.kwd_to_binary(kwd_path, bin_path, chan_list=chan_list)
    print('done')
#make all the parameters for sorting with kilosort

# sort with a script in niao (scripts gets the data from cube, puts back in cube)
def niao_kilosort(bird, sess):
    print('Sending sort to niao')
    return ssh_paramiko('niao', 'py_kilosort.sh {0} {1}'.format(bird, sess))

def local_kilosort(bird, sess):
    logger.info('Will sort using kilosort on local computer')
    kilosort.run_kilosort(bird, sess, 
                          kilo_dir=os.path.abspath('/home/earneodo/repos/KiloSort'),
                          npymat_dir=os.path.abspath('/home/earneodo/repos/npy-matlab'))

#convert back to kwik file

2017-06-02 13:08:54,818 root         INFO     Computer: passaro
2017-06-02 13:08:54,885 root         INFO     All modules loaded


In [3]:
bird_id = 'z030'
sess_day = ['2016-12-16']
depth = '100'
raw_location = 'rw' #raw in local, rw in cube
# Make the super session and gather the list of sessions
# get the sessions list of files
raw_data_folder = et.file_names(bird_id)['folders'][raw_location]

all_sessions = [glob.glob(os.path.join(raw_data_folder, sess_day + '*' + str(depth))) for sess_day in sess_days]
sessions = [s for sl in all_sessions for s in sl]
day_par_file_path = os.path.join(sessions[0], et.file_names(bird_id)['structure']['par'])
sess_par = et.get_parameters(bird_id, os.path.split(sessions[0])[-1], location=raw_location)
super_sess_par = sess_par.copy()
data_processor = sess_par['rec_config']['processors']['data']
experiments = ss.list_flatten([glob.glob(os.path.join(s, '*_{}.raw.kwd'.format(data_processor)))[:] for s in sessions ])
experiments.sort()
print(experiments)

IndexError: list index out of range

## Make The supersession

In [2]:
# Do the supersess
reload(ss)
bird_id = 'z007'
depth = '100'
sess_base_id = '2016-09-09'
raw_location = 'rw'
super_sess_name = ss.process_awake_recording(bird_id, sess_base_id, depth, 
                                             cond='*', raw_location=raw_location)

2017-05-24 18:25:49,136 supersession INFO     Super session path /mnt/cube/earneodo/bci_zf/ss_data/z007/day-all_2016-09-09_100/experiment.raw.kwd
2017-05-24 18:25:49,155 supersession INFO     searching *_100.raw.kwd
2017-05-24 18:25:49,175 supersession INFO     Making supersession day-all_2016-09-09_100
2017-05-24 18:25:49,179 supersession INFO     super file path: /mnt/cube/earneodo/bci_zf/ss_data/z007/day-all_2016-09-09_100/experiment.raw.kwd
2017-05-24 18:25:49,182 supersession INFO     Found 14 experiment files
2017-05-24 18:25:49,184 bci.core.expstruct DEBUG    Creating directory /mnt/cube/earneodo/bci_zf/ss_data/z007/day-all_2016-09-09_100
2017-05-24 18:25:49,197 supersession INFO     Inserting file /mnt/cube/earneodo/bci_zf/raw_data/z007/2016-09-09_08-48-48_100/experiment1_100.raw.kwd
2017-05-24 18:25:49,228 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r+)>


['/mnt/cube/earneodo/bci_zf/raw_data/z007/sleep_2016-09-09_00-57-51_100', '/mnt/cube/earneodo/bci_zf/raw_data/z007/sleep_2016-09-09_01-57-58_100', '/mnt/cube/earneodo/bci_zf/raw_data/z007/2016-09-09_14-58-10_100', '/mnt/cube/earneodo/bci_zf/raw_data/z007/2016-09-09_17-12-20_100', '/mnt/cube/earneodo/bci_zf/raw_data/z007/sleep_2016-09-09_02-58-06_100', '/mnt/cube/earneodo/bci_zf/raw_data/z007/2016-09-09_16-10-46_100', '/mnt/cube/earneodo/bci_zf/raw_data/z007/2016-09-09_14-22-15_100', '/mnt/cube/earneodo/bci_zf/raw_data/z007/2016-09-09_09-55-46_100', '/mnt/cube/earneodo/bci_zf/raw_data/z007/sleep_2016-09-09_03-58-13_100', '/mnt/cube/earneodo/bci_zf/raw_data/z007/2016-09-09_11-12-35_100', '/mnt/cube/earneodo/bci_zf/raw_data/z007/2016-09-09_17-34-37_100', '/mnt/cube/earneodo/bci_zf/raw_data/z007/2016-09-09_08-48-48_100', '/mnt/cube/earneodo/bci_zf/raw_data/z007/sleep_2016-09-09_05-58-27_100', '/mnt/cube/earneodo/bci_zf/raw_data/z007/sleep_2016-09-09_04-58-20_100']
['/mnt/cube/earneodo/bci_

2017-05-24 18:25:59,019 root         DEBUG    H5 file: <HDF5 file "experiment.lfp.kwd" (mode r+)>
2017-05-24 18:25:59,024 supersession DEBUG    frame size 19200
/usr/local/anaconda/lib/python2.7/site-packages/scipy/signal/signaltools.py:3398: FutureWarning:  Note: Decimate's zero_phase keyword argument will default to True in a future release. Until then, decimate defaults to one-way filtering for backwards compatibility. Ideally, always set this argument explicitly.
  "explicitly.", FutureWarning)
2017-05-24 18:26:14,828 supersession INFO     Inserting file /mnt/cube/earneodo/bci_zf/raw_data/z007/2016-09-09_09-55-46_100/experiment0_100.raw.kwd
2017-05-24 18:26:15,176 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r+)>
2017-05-24 18:26:15,178 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r+)>
2017-05-24 18:26:24,786 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r+)>
2017-05-24 18:26:24,788 root         DEBUG    H5 file

2017-05-24 18:50:07,406 root         DEBUG    H5 file: <HDF5 file "experiment.lfp.kwd" (mode r+)>
2017-05-24 18:50:07,407 root         DEBUG    H5 file: <HDF5 file "experiment.lfp.kwd" (mode r+)>
2017-05-24 18:50:07,411 supersession DEBUG    frame size 19200
2017-05-24 18:51:28,212 supersession INFO     Inserting file /mnt/cube/earneodo/bci_zf/raw_data/z007/sleep_2016-09-09_04-58-20_100/experiment1_100.raw.kwd
2017-05-24 18:51:28,333 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r+)>
2017-05-24 18:51:28,335 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r+)>
2017-05-24 18:53:50,290 root         DEBUG    H5 file: <HDF5 file "experiment.lfp.kwd" (mode r+)>
2017-05-24 18:53:50,293 root         DEBUG    H5 file: <HDF5 file "experiment.lfp.kwd" (mode r+)>
2017-05-24 18:53:50,298 supersession DEBUG    frame size 19200
2017-05-24 18:55:12,505 supersession INFO     Inserting file /mnt/cube/earneodo/bci_zf/raw_data/z007/sleep_2016-09-09_05-58-27_100/

In [16]:
super_sess_name

'day-all_2016-09-09_08_100'

# Kilosort

### On Niao

In [5]:
# Do the kilosort
#super_sess_name = 'day-*2016-09-09_08_100'
#super_sess_name = 'day-' + super_sess_name + '_' + depth
print "making binary"
make_binary(bird_id, super_sess_name)
reload(kilosort)
#reload(kwkf)
print 'sorting in niao'
niao_msg = niao_kilosort(bird_id, super_sess_name)
print niao_msg
print 'getting back to kwik format'
kwkf.kilo_to_kwik(bird_id, super_sess_name)

2017-05-25 08:24:56,029 paramiko.transport DEBUG    starting thread (client mode): 0x136b8910L
2017-05-25 08:24:56,031 paramiko.transport DEBUG    Local version/idstring: SSH-2.0-paramiko_2.1.2
2017-05-25 08:24:56,043 paramiko.transport DEBUG    Remote version/idstring: SSH-2.0-OpenSSH_6.6.1p1 Ubuntu-2ubuntu2.8
2017-05-25 08:24:56,044 paramiko.transport INFO     Connected (version 2.0, client OpenSSH_6.6.1p1)
2017-05-25 08:24:56,047 paramiko.transport DEBUG    kex algos:[u'curve25519-sha256@libssh.org', u'ecdh-sha2-nistp256', u'ecdh-sha2-nistp384', u'ecdh-sha2-nistp521', u'diffie-hellman-group-exchange-sha256', u'diffie-hellman-group-exchange-sha1', u'diffie-hellman-group14-sha1', u'diffie-hellman-group1-sha1'] server key:[u'ssh-rsa', u'ssh-dss', u'ecdsa-sha2-nistp256'] client encrypt:[u'aes128-ctr', u'aes192-ctr', u'aes256-ctr', u'arcfour256', u'arcfour128', u'aes128-gcm@openssh.com', u'aes256-gcm@openssh.com', u'chacha20-poly1305@openssh.com', u'aes128-cbc', u'3des-cbc', u'blowfish-c

sorting in niao
Sending sort to niao
Connected to niao


2017-05-25 08:24:57,802 paramiko.transport DEBUG    [chan 0] Max packet out: 32768 bytes
2017-05-25 08:24:57,805 paramiko.transport DEBUG    Secsh channel 0 opened.
2017-05-25 08:24:57,808 paramiko.transport DEBUG    [chan 0] Sesch channel 0 request ok


None


2017-05-25 11:38:57,645 paramiko.transport DEBUG    [chan 0] EOF received (0)
2017-05-25 11:38:57,649 kwik_functions INFO     Creating kwik file for bird: z007 sess: day-all_2016-09-09_100
2017-05-25 11:38:57,652 kwik_functions INFO     Found clu file, will attempt to unpack manual sorted data from kilosort
2017-05-25 11:38:57,654 kwik_functions DEBUG    {'par': '/mnt/cube/earneodo/bci_zf/ss_data/z007/day-all_2016-09-09_100/params.py', 'spk': '/mnt/cube/earneodo/bci_zf/ss_data/z007/day-all_2016-09-09_100/spike_times.npy', 'kwd': '/mnt/cube/earneodo/bci_zf/ss_data/z007/day-all_2016-09-09_100/experiment.raw.kwd', 'grp': '/mnt/cube/earneodo/bci_zf/ss_data/z007/day-all_2016-09-09_100/cluster_groups.csv', 'clu': '/mnt/cube/earneodo/bci_zf/ss_data/z007/day-all_2016-09-09_100/spike_clusters.npy', 'temp': None, 'kwk': '/mnt/cube/earneodo/bci_zf/ss_data/z007/day-all_2016-09-09_100/experiment.kwik'}


finished command, returning
['/mnt/cube/earneodo/bci_zf/templates\nFinished \n']
getting back to kwik format
No grp file found


2017-05-25 11:38:59,209 kwik_functions INFO     Making spike tables
2017-05-25 11:38:59,212 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 11:38:59,213 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 11:38:59,214 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 11:38:59,215 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 11:38:59,216 root         DEBUG    Getting dataset from rec 0
2017-05-25 11:38:59,218 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 11:38:59,219 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 11:38:59,220 root         DEBUG    Getting dataset from rec 1
2017-05-25 11:38:59,221 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 11:38:59,222 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25

2017-05-25 11:39:03,651 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 11:39:03,652 root         DEBUG    Getting dataset from rec 11
2017-05-25 11:39:03,653 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 11:39:03,654 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 11:39:03,655 root         DEBUG    Getting dataset from rec 12
2017-05-25 11:39:03,657 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 11:39:03,658 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 11:39:03,659 root         DEBUG    Getting dataset from rec 13
2017-05-25 11:39:03,661 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 11:39:03,662 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 11:39:03,663 root         DEBUG    Getting dataset from rec 14
2017-05-25 11:39:03,664 r

### On Passaro

In [7]:
reload(kilosort)
print "making binary"
make_binary(bird_id, super_sess_name)
def local_kilosort(bird, sess):
    logger.info('Will sort using kilosort on local computer')
    kilosort.run_kilosort(bird, sess, 
                          kilo_dir=os.path.abspath('/home/earneodo/repos/KiloSort'),
                          npymat_dir=os.path.abspath('/home/earneodo/repos/npy-matlab'),
                          no_copy=False,
                          use_gpu=False)

passaro_msg = local_kilosort(bird_id, super_sess_name)

2017-05-24 16:47:59,457 root         INFO     Will sort using kilosort on local computer
2017-05-24 16:47:59,459 kilosort     INFO     will run bci_pipeline on bird z007 - session day-all_2016-09-09_08_100
2017-05-24 16:47:59,463 kilosort     INFO     Saving output in log file: /mnt/cube/earneodo/bci_zf/ss_data/z007/day-all_2016-09-09_08_100/kilosort_py.log
2017-05-24 16:47:59,464 kilosort     INFO     will run bci_pipeline on bird z007 - session day-all_2016-09-09_08_100
2017-05-24 16:47:59,466 kilosort     INFO     Will not pull data from cube
2017-05-24 16:47:59,467 kilosort     INFO     Will create the scripts
2017-05-24 16:47:59,486 kilosort     DEBUG    local sort dir: /usr/local/experiment/tmp_data/z007/day-all_2016-09-09_08_100
2017-05-24 16:47:59,489 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-24 16:47:59,491 kilosort     DEBUG    n_chan: 32
2017-05-24 16:47:59,492 kilosort     DEBUG    {'Nfilt': 64, 'datadir': '/usr/local/experiment/tmp_da

/mnt/cube/earneodo/bci_zf/templates


2017-05-24 16:58:32,889 kilosort     INFO     Sorted MATLAB is selecting SOFTWARE OPENGL rendering.

                            < M A T L A B (R) >
                  Copyright 1984-2017 The MathWorks, Inc.
                   R2017a (9.2.0.538062) 64-bit (glnxa64)
                             February 23, 2017

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 
    ----------------------------------------------------
	Your MATLAB license will expire in 38 days.
	Please contact your system administrator or
	MathWorks to renew this license.
    ----------------------------------------------------

.               chanMap.mat     experiment.dat  temp_wh.dat     
..              config.m        master.m        

/home/earneodo/repos/KiloSort

fpath =

    '/usr/local/experiment/tmp_data/z007/day-all_2016-09-09_08_100'


ops = 

  struct with fields:

            GPU: 0
         parfor: 1
        verbose: 1
    showfigures: 1

2017-05-24 16:58:32,924 kilosort     INFO     Pushing back bci_pipeline data
2017-05-24 16:58:33,513 kilosort     INFO     Pushed back to cube [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


Finished 


# STIMULUS ALIGNMENT

In [74]:
from bci import stimalign as sa

In [3]:
super_sess_name = 'day-all_2016-09-09_100'
bird_id = 'z007'

In [5]:
reload(et)
et.get_locations()

{'archive': '/mnt/cube/earneodo/archive/bci_zf',
 'experiment': '/mnt/cube/earneodo/bci_zf',
 'experiment_local': '/usr/local/experiment',
 'repos': '/mnt/cube/earneodo/repos',
 'scratch': '/usr/local/experiment/scratchpad',
 'store': '/Data/bci_zf'}

In [17]:
reload(sa)
sa.align_stim(bird_id, super_sess_name)

2017-05-25 13:47:44,034 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 13:47:44,036 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 13:47:44,037 root         DEBUG    Getting group of rec 0
2017-05-25 13:47:44,039 stimalign    DEBUG    Rec origin: {'rec': 0, 'sess': '2016-09-09_08-48-48_100', 'bird': 'z007', 'structure': 'experiment1_100'}
2017-05-25 13:47:44,064 stimalign    INFO     Empty list of play_wav in file /mnt/cube/earneodo/bci_zf/raw_data/z007/2016-09-09_08-48-48_100/experiment1.kwe
2017-05-25 13:47:44,067 stimalign    DEBUG    Stim_names: []
2017-05-25 13:47:44,071 stimalign    INFO     Done
2017-05-25 13:47:44,074 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 13:47:44,076 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 13:47:44,078 root         DEBUG    Getting group of rec 1
2017-05-25 13:47:44,081 stimalign    DEBUG    Rec ori

2017-05-25 13:47:48,241 stimalign    DEBUG    Stim_names: ['bos_rev_tag' 'bos_syn_tag' 'bos_tag']
2017-05-25 13:47:48,242 stimalign    INFO     Getting starts of stim bos_rev_tag
2017-05-25 13:47:48,244 root         DEBUG    H5 file: <HDF5 file "experiment1_100.raw.kwd" (mode r)>
2017-05-25 13:47:48,244 root         DEBUG    Getting dataset from rec 0
2017-05-25 13:47:48,487 stimalign    INFO     Found 17 starts
2017-05-25 13:47:48,492 stimalign    INFO     Stored in /mnt/cube/earneodo/bci_zf/ss_data/z007/day-all_2016-09-09_100/experiment.sng.kwe
2017-05-25 13:47:48,495 stimalign    INFO     Getting starts of stim bos_syn_tag
2017-05-25 13:47:48,496 root         DEBUG    H5 file: <HDF5 file "experiment1_100.raw.kwd" (mode r)>
2017-05-25 13:47:48,497 root         DEBUG    Getting dataset from rec 0
2017-05-25 13:47:49,486 stimalign    INFO     Found 16 starts
2017-05-25 13:47:49,489 stimalign    INFO     Stored in /mnt/cube/earneodo/bci_zf/ss_data/z007/day-all_2016-09-09_100/experiment.

2017-05-25 13:47:53,478 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 13:47:53,480 root         DEBUG    Getting group of rec 6
2017-05-25 13:47:53,481 stimalign    DEBUG    Rec origin: {'rec': 0, 'sess': '2016-09-09_14-58-10_100', 'bird': 'z007', 'structure': 'experiment1_100'}
2017-05-25 13:47:53,493 stimalign    INFO     Empty list of play_wav in file /mnt/cube/earneodo/bci_zf/raw_data/z007/2016-09-09_14-58-10_100/experiment1.kwe
2017-05-25 13:47:53,494 stimalign    DEBUG    Stim_names: []
2017-05-25 13:47:53,495 stimalign    INFO     Done
2017-05-25 13:47:53,497 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 13:47:53,498 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-05-25 13:47:53,499 root         DEBUG    Getting group of rec 7
2017-05-25 13:47:53,500 stimalign    DEBUG    Rec origin: {'rec': 0, 'sess': '2016-09-09_16-10-46_100', 'bird': 'z007', 'structure': 'experiment1_100'

# COPY SOME RAW FILES TO SS_FOLDER

In [114]:
import re
import shutil

def copy_files(src_fold, dest_fold, f_list):
    assert(os.path.isdir(src_fold))
    for f in f_list:
        #logger.debug(os.path.join(src_fold, f[0]))
        #logger.debug(os.path.join(dest_fold, f[1]))
        shutil.copyfile(os.path.join(src_fold, f[0]), os.path.join(dest_fold, f[1]))

def strip_time(sess_str):
    m = re.search(r'_\d{2}-\d{2}-\d{2}', sess_str)
    time_str = m.group()
    return time_str[1:], sess_str.replace(time_str, '')

def split_path(path):
    folders = []
    while 1:
        path, folder = os.path.split(path)
        if folder != '':
            folders.append(folder)
        else:
            if path != '':
                folders.append(path)
            break
            
    folders.reverse()
    return folders

In [137]:
bird_id = 'z034'
sess_days = ['*2017-04-21']
depth = ''
raw_location = 'rw' #raw in local, rw in cube

files_to_copy = [['experiment.par.yml', 'experient.par.raw.yml'],
              ['settings.xml', 'settings.raw.xml']]

# Make the super session and gather the list of sessions
# get the sessions list of files
raw_data_folder = et.file_names(bird_id)['folders'][raw_location]

print(sess_day)
search_str = os.path.join(raw_data_folder, sess_day + '*' + str(depth))
all_sessions = [glob.glob(os.path.join(raw_data_folder, sess_day + '*' + str(depth))) for sess_day in sess_days]
sessions = [s for sl in all_sessions for s in sl]
day_par_file_path = os.path.join(sessions[0], et.file_names(bird_id)['structure']['par'])
sess_par = et.get_parameters(bird_id, os.path.split(sessions[0])[-1], location=raw_location)
super_sess_par = sess_par.copy()
data_processor = sess_par['rec_config']['processors']['data']


experiments = ss.list_flatten([glob.glob(os.path.join(s, '*_{}.raw.kwd'.format(data_processor)))[:] for s in sessions ])
experiments.sort()
print(experiments)

*2017-04-24
['/mnt/cube/earneodo/bci_zf/raw_data/z034/anesth_surface_2017-04-21_14-07-50_0/experiment1_100.raw.kwd', '/mnt/cube/earneodo/bci_zf/raw_data/z034/anesth_surface_2017-04-21_14-50-15_1/experiment1_100.raw.kwd', '/mnt/cube/earneodo/bci_zf/raw_data/z034/anesth_surface_2017-04-21_19-00-19_5/experiment1_100.raw.kwd', '/mnt/cube/earneodo/bci_zf/raw_data/z034/anesth_surface_deep_2017-04-21_16-01-36_650/experiment1_100.raw.kwd', '/mnt/cube/earneodo/bci_zf/raw_data/z034/anesth_surface_depth_2017-04-21_20-14-53_655/experiment1_100.raw.kwd', '/mnt/cube/earneodo/bci_zf/raw_data/z034/explore_2017-04-21_12-10-16_0/experiment1_100.raw.kwd', '/mnt/cube/earneodo/bci_zf/raw_data/z034/explore_2017-04-21_12-48-11_1/experiment1_100.raw.kwd', '/mnt/cube/earneodo/bci_zf/raw_data/z034/explore_2017-04-21_13-59-21_2/experiment1_100.raw.kwd', '/mnt/cube/earneodo/bci_zf/raw_data/z034/explore_2017-04-21_18-47-21_6/experiment1_100.raw.kwd', '/mnt/cube/earneodo/bci_zf/raw_data/z034/explore_2017-04-21_18-5

In [138]:
experiments

['/mnt/cube/earneodo/bci_zf/raw_data/z034/anesth_surface_2017-04-21_14-07-50_0/experiment1_100.raw.kwd',
 '/mnt/cube/earneodo/bci_zf/raw_data/z034/anesth_surface_2017-04-21_14-50-15_1/experiment1_100.raw.kwd',
 '/mnt/cube/earneodo/bci_zf/raw_data/z034/anesth_surface_2017-04-21_19-00-19_5/experiment1_100.raw.kwd',
 '/mnt/cube/earneodo/bci_zf/raw_data/z034/anesth_surface_deep_2017-04-21_16-01-36_650/experiment1_100.raw.kwd',
 '/mnt/cube/earneodo/bci_zf/raw_data/z034/anesth_surface_depth_2017-04-21_20-14-53_655/experiment1_100.raw.kwd',
 '/mnt/cube/earneodo/bci_zf/raw_data/z034/explore_2017-04-21_12-10-16_0/experiment1_100.raw.kwd',
 '/mnt/cube/earneodo/bci_zf/raw_data/z034/explore_2017-04-21_12-48-11_1/experiment1_100.raw.kwd',
 '/mnt/cube/earneodo/bci_zf/raw_data/z034/explore_2017-04-21_13-59-21_2/experiment1_100.raw.kwd',
 '/mnt/cube/earneodo/bci_zf/raw_data/z034/explore_2017-04-21_18-47-21_6/experiment1_100.raw.kwd',
 '/mnt/cube/earneodo/bci_zf/raw_data/z034/explore_2017-04-21_18-51-5

In [139]:
one_exp = experiments[0]

def copy_raw_meta_files(one_exp, files_to_copy, raw_location='rw'):
    raw_sess = split_path(one_exp)[-2]
    _, ss_sess = strip_time(raw_sess)

    ss_fn = et.file_names(bird_id, 'day-{}'.format(ss_sess))
    raw_fn = et.file_names(bird_id, raw_sess)

    ss_fold = ss_fn['folders']['ss']
    raw_fold = raw_fn['folders'][raw_location]

    if os.path.isdir(ss_fold):
        # copy the files
        logger.info('Will copy raw metadata files of {}'.format(ss_fold))
        copy_files(raw_fold, ss_fold, files_to_copy)
    else:
        logger.debug('No ss data folder {}, skipping'.format(ss_fold))
#ss_sess_fold = 

for one_exp in experiments:
    copy_raw_meta_files(one_exp, files_to_copy, raw_location=raw_location)

2017-06-02 14:35:01,754 root         INFO     Will copy raw metadata files of /mnt/cube/earneodo/bci_zf/ss_data/z034/day-anesth_surface_2017-04-21_0
2017-06-02 14:35:01,833 root         INFO     Will copy raw metadata files of /mnt/cube/earneodo/bci_zf/ss_data/z034/day-anesth_surface_2017-04-21_1
2017-06-02 14:35:01,896 root         INFO     Will copy raw metadata files of /mnt/cube/earneodo/bci_zf/ss_data/z034/day-anesth_surface_2017-04-21_5
2017-06-02 14:35:01,942 root         DEBUG    No ss data folder /mnt/cube/earneodo/bci_zf/ss_data/z034/day-anesth_surface_deep_2017-04-21_650, skipping
2017-06-02 14:35:01,945 root         DEBUG    No ss data folder /mnt/cube/earneodo/bci_zf/ss_data/z034/day-anesth_surface_depth_2017-04-21_655, skipping
2017-06-02 14:35:01,947 root         DEBUG    No ss data folder /mnt/cube/earneodo/bci_zf/ss_data/z034/day-explore_2017-04-21_0, skipping
2017-06-02 14:35:01,950 root         DEBUG    No ss data folder /mnt/cube/earneodo/bci_zf/ss_data/z034/day-exp

In [121]:
files_to_copy = [['experiment.par.yml', 'experient.par.raw.yml'],
              ['settings.xml', 'settings.raw.xml']]


In [117]:
logger.debug

'/mnt/cube/earneodo/bci_zf/ss_data/z030/day_anesth_deep_2016-12-16_450'

In [101]:


split_path(one_exp)
strip_time(raw_sess_fold)

('17-51-06', 'anesth_deep_2016-12-16_17-51-06_450')

In [71]:
os.path.isdir(one_fn['folders']['ss'])

False